# This notebook  takes the list of urls from the previous scrape to then scrape and save the whole html file of each human forecast to a mongoDB collection. Then it parses through the html using beautiful soup

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from selenium.webdriver import Chrome
import pymongo
import time
import pandas as pd
from bs4 import BeautifulSoup
import json
import scrape_soup as s_soup

## import scrape_soup with the function and other module imports

## do not initiate another shadow nor close the first one that opens when the module loads!

In [12]:
mc = pymongo.MongoClient()
db = mc['avalanche']
forecast_coll = db['forecasts']

In [4]:
urls_df = pd.read_json('json_archive/url_scrape.json')

In [8]:
urls =list(urls_df['urls'])
len(urls)

346

In [9]:
for url in urls:
    s_soup.collect_page(url)

In [10]:
db.list_collection_names()

['forecasts_test', 'stevens_forecasts', 'forecasts']

In [ ]:
db.forecasts.find_one()

In [13]:
forecast_coll.count_documents({})

347

## To save time, I want to isolate the stevens pass urls, but how? 

https://stackoverflow.com/questions/10242501/how-to-find-a-substring-in-a-field-in-mongodb

db.test.find({A: {$regex: 'Star Wars'}})


In [14]:
coll = forecast_coll #makes it easier to type

In [15]:
query= {'url': 'https://www.nwac.us/avalanche-forecast/'
           'avalanche-region-forecast/5176/cascade-west-stevens-pass/'}

In [16]:
first_cur = coll.find(query)

In [17]:
coll.count_documents(query)

1

I can find the first one, now to find all that are stevens pass


In [19]:
cur = coll.find({})
stevens = [forecasts for forecasts in cur if 'stevens' in forecasts['url']]

In [20]:
len(stevens)

36

In [21]:
type(stevens)

list

In [22]:
stevens[0]['html']

'<html class=" overthrow-enabled"><head>\n        <meta http-equiv="content-type" content="text/html; charset=utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n        <meta name="description" content="Provides mountain weather and avalanche information for the Northwest US, including the Washington Cascades, Olympic Mountains and the northern Oregon Cascades.">\n<!--        <link rel="image_src" href="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg"> -->\n        <link rel="image_src" href="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg">\n\n\n        <meta property="og:type" content="website">\n<!--         <meta property="og:image" content="http://www.nwac.us/static/common/images/nwac-logo-square-200.jpg" /> -->\n        <meta property="og:image" content="//www.nwac.us/static/common/images/nwac-logo-square-200.jpg">\n        <meta property="og:image:type" content="image/jpeg">\n<!--         <meta property="og:url

In [ ]:
soup = BeautifulSoup(stevens[0]['html'],'lxml')

In [ ]:
#print(soup.prettify())

In [ ]:
#main-content > div.forecast-info > div.forecast-date
#elevation-levels-header > div.span4.desktop.elevation-day-name > p

In [ ]:
danger_above_tag = '#treeline-above > div.span4.elev-day1-column > div.danger-description > h4'
danger_near_tag ='#treeline-near > div.span4.elev-day1-column > div.danger-description > h4'
danger_below_tag = '#treeline-below > div.span4.elev-day1-column > div.danger-description > h4'
area_tag = '#main-content > h2'
date_tomorrow_tag = '#elevation-levels-header > div.span4.desktop.elevation-day-name > p'
date_today_tag = '#main-content > div.forecast-info > div.forecast-date'

In [ ]:
def create_danger_info(page):
    '''Collects the relevant info from the webpage html'''
    
    
    danger_above_tag = '#treeline-above > div.span4.elev-day1-column > div.danger-description > h4'
    danger_near_tag ='#treeline-near > div.span4.elev-day1-column > div.danger-description > h4'
    danger_below_tag = '#treeline-below > div.span4.elev-day1-column > div.danger-description > h4'
    area_tag = '#main-content > h2'
    date_tomorrow_tag = '#elevation-levels-header > div.span4.desktop.elevation-day-name > p'
    #date_today_tag = '#main-content > div.forecast-info > div.forecast-date'
    soup = BeautifulSoup(page['html'],'lxml')
   
    area = soup.select_one(area_tag).text
    date_tomorrow = soup.select_one(date_tomorrow_tag).text
    #date_today = soup.select_one(date_today_tag).text #its in a string with other text, not very useful, can engineer this later
    danger_above = soup.select_one(danger_above_tag).text
    danger_near = soup.select_one(danger_near_tag).text
    danger_below = soup.select_one(danger_below_tag).text
    return {
            'date_tomorrow': date_tomorrow,
            #'date_today': date_today,
            'area': area,
            'danger_above_treeline': danger_above,
            'danger_near_treeline': danger_near,
            'danger_below_treeline': danger_below,
            }

In [ ]:
soup = BeautifulSoup(stevens[0]['html'],'lxml')

In [ ]:
create_danger_info(stevens[0])

In [ ]:
avalanche_forcasts = [create_danger_info(page) for page in stevens]
avalanches = pd.DataFrame(avalanche_forcasts)


In [ ]:
avalanches = avalanches.drop_duplicates()

In [ ]:
avalanches.to_json('stevens_avalanches.json')

In [ ]:
df = pd.read_json('stevens_avalanches.json')


In [ ]:
df.shape

In [ ]:
for page in stevens:
    create_danger_list(page)
    
    

In [ ]:
danger_above = soup.select_one(danger_above_tag).text
display(danger_above)
area_forecast = soup.select_one(area_tag).text
display(area_forecast)
date_forecast = soup.select_one(date_tag).text
display(date_forecast)
danger_above = soup.select_one(danger_above_tag).text
display(danger_above)
danger_near = soup.select_one(danger_near_tag).text
display(danger_near)
danger_below = soup.select_one(danger_below_tag).text
display(danger_below)

In [ ]:
danger_above_tag = '#treeline-above > div.span4.elev-day1-column > div.danger-description > h4'
danger_near_tag ='#treeline-near > div.span4.elev-day1-column > div.danger-description > h4'
danger_below_tag = '#treeline-below > div.span4.elev-day1-column > div.danger-description > h4'
area_tag = '#main-content > h2'
date_tag = '#elevation-levels-header > div.span4.desktop.elevation-day-name > p'

result = []
soup = BeautifulSoup(stevens[0]['html'],'lxml')

area_forecast = soup.select_one(area_tag).text
date_forecast = soup.select_one(date_tag).text
danger_above = soup.select_one(danger_above_tag).text
danger_near = soup.select_one(danger_near_tag).text
danger_below = soup.select_one(danger_below_tag).text
result.append({'date': date_forecast,
               'area': area_forecast,
               'danger_above_treeline': danger_above,
               'danger_near_treeline': danger_near,
               'danger_below_treeline': danger_below,

              })

In [ ]:
result

In [ ]:
len(htmls)

In [ ]:
for selector in selectors:
    print({selector[10:15]:soup.select(selector)[0].text})

In [ ]:
stevens[0]['url']

In [ ]:
for page in stevens[:2]:
        

In [ ]:
textabove = soup.select_one('#elevation-levels-header > div.span4.desktop.elevation-day-name > p')
textabove.text

In [ ]:
soup.select_one('#treeline-near > div.span4.elev-day1-column > div.danger-description > h4').text

In [ ]:
soup.select_one('#treeline-below > div.span4.elev-day1-column > div.danger-description > h4').text

In [ ]:
soup.select_one('#treeline-above > div.span4.elev-day1-column > div.danger-description > h4').text